# Append Data
Reads a csv into a dataframe, chunks the dataframe, publishes the first chunk, then appends the remaining chunks. This showcases the **[`edit_features`](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.features.toc.html#arcgis.features.FeatureLayer.edit_features)** function to append data when working with a [SpatialDataFrame](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.features.toc.html#arcgis.features.SpatialDataFrame)

In [ ]:
import pandas as pd
from arcgis.gis import GIS

***Note**, if you are unable to import local `partnerutils`, **copy the following functions** from [`cool_utils`](https://github.com/mpayson/esri-partner-tools/blob/master/partnerutils/cool_utils.py) and [`feature_utils`](https://github.com/mpayson/esri-partner-tools/blob/master/partnerutils/feature_utils.py)

In [ ]:
from partnerutils.cool_utils import chunk_df
from partnerutils.feature_utils import sdf_from_xyz

In [ ]:
# log in to your GIS
gis = GIS(username="mpayson_startups")

In [ ]:
# path to data
csv_path = "../test_data/NYC Inspections Geocoded.csv"

x_col = "x"
y_col = "y"

In [ ]:
# read csv and construct spatial dataframe
df = pd.read_csv(csv_path)
sdf = sdf_from_xyz(df, x_col, y_col)
len(sdf)

In [ ]:
# iterate through chunks to create and append data
lyr = None
for c_df in chunk_df(sdf, 50):
    if not lyr:
        item = c_df.to_featurelayer("MyFeatureService")
        lyr = item.layers[0]
    else:
        # THIS IS THE APPEND DATA PART
        fs = c_df.to_featureset()
        success = lyr.edit_features(adds=fs)
item